In [1]:
import pandas as pd
import numpy as np
import py_stringmatching as sm
import py_stringsimjoin as ssj
from pandas_profiling import ProfileReport
import re
import nltk
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
ban = pd.read_csv('../../datasets/books/ban.csv',dtype='str',sep=';')

In [3]:
half = pd.read_csv('../../datasets/books/half.csv',dtype='str',sep=';')

In [4]:
ban.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17629 entries, 0 to 17628
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ban_id                          17629 non-null  object
 1   ban_isbn                        17629 non-null  object
 2   ban_title                       17628 non-null  object
 3   ban_author                      16277 non-null  object
 4   ban_binding                     2961 non-null   object
 5   ban_pubdate                     8822 non-null   object
 6   ban_pages                       17629 non-null  object
 7   ban_publisher                   17589 non-null  object
 8   ban_title_no_par                17628 non-null  object
 9   ban_title_no_par_no_stop_words  17615 non-null  object
 10  ban_firstauthor                 16277 non-null  object
 11  ban_firsttwoauthors             16277 non-null  object
dtypes: object(12)
memory usage: 1.6+ MB


In [5]:
half.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3099 entries, 0 to 3098
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   half_id                          3099 non-null   object
 1   half_isbn                        3099 non-null   object
 2   half_title                       3099 non-null   object
 3   half_firstauthor                 3094 non-null   object
 4   half_binding                     460 non-null    object
 5   half_pubdate                     3028 non-null   object
 6   half_pages                       3099 non-null   object
 7   half_publisher                   3099 non-null   object
 8   half_title_no_par                3099 non-null   object
 9   half_title_no_par_no_stop_words  3094 non-null   object
dtypes: object(10)
memory usage: 242.2+ KB


In [6]:
ban.fillna('',inplace=True)
half.fillna('',inplace=True)

**-- Half only contains first authors**

### Join on author (ban_author, half_author) and (ban_firstauthor, half_author) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [9]:
q3 = sm.QgramTokenizer(qval=3,return_set=True)

In [10]:
ban_half_pairs_a_q3_jac_03 = ssj.jaccard_join(ban, half, 'ban_id', 'half_id', 'ban_author_lower', 'half_author_lower', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower', 'ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

/home/debian/anaconda3/envs/mag/lib/python3.6/site-packages/py_stringsimjoin/utils/validation.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if attr_type != pd.np.object:


In [11]:
ban_half_pairs_fa_q3_jac_03 = ssj.jaccard_join(ban, half, 'ban_id', 'half_id', 'ban_firstauthor', 'half_author_lower', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower', 'ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Join on title (with parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [13]:
ban_half_pairs_tit_q3_jac_03 = ssj.jaccard_join(ban, half, 'ban_id', 'half_id', 'ban_title_lower', 'half_title_lower', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower', 'ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Join on title (without parentheses) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [14]:
ban_half_pairs_titnp_q3_jac_03 = ssj.jaccard_join(ban, half, 'ban_id', 'half_id', 'ban_title_no_par', 'half_title_no_par', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower', 'ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Join on title (without parentheses and stop words) based on Jaccard score of at least 0.3 with 3-gram tokenization

In [15]:
ban_half_pairs_titnp_nosw_q3_jac_03 = ssj.jaccard_join(ban, half, 'ban_id', 'half_id', 'ban_title_no_par', 'half_title_no_par', q3, 0.3, 
                                     l_out_attrs=['ban_title','ban_title_lower','ban_title_no_par','ban_title_no_par_no_stop_words','ban_author','ban_author_lower', 'ban_firstauthor', 'ban_firsttwoauthors','ban_binding','ban_pubdate', 'ban_pages','ban_publisher','ban_isbn'], 
                                     r_out_attrs=['half_isbn','half_title','half_title_lower','half_title_no_par','half_title_no_par_no_stop_words','half_author','half_author_lower','half_binding','half_pubdate','half_pages','half_publisher'],
                                     n_jobs=-3)

### Evaluate Blocking

In [17]:
#Calculafte the true matches (based on ISBN) and store it in matches_ban_half
matches_ban_half = pd.merge(ban,half,left_on='ban_isbn',right_on='half_isbn')
print('True matches across Ban and Half: {}'.format(matches_ban_half.shape[0]))

True matches across Ban and Half: 886


In [18]:
def assignTrueLabels(df_blockedpairs, idcolumn1, idcolumn2):
    return df_blockedpairs.apply(lambda row: 1 if (row[idcolumn1]==row[idcolumn2]) else 0, axis=1)

In [19]:
def calcRedRatio(df1,df2,df_blockedpairs):
    cp = (df1.shape[0]*df2.shape[0])/2
    rr = (1-(df_blockedpairs.shape[0]/cp))*100
    return round(rr,2)

In [20]:
def calcPairCompISBN(df_matches,df_blockedpairs,true_label_col):
    pc = (df_blockedpairs[df_blockedpairs[true_label_col]==1].shape[0]/df_matches.shape[0])*100
    return round(pc,2)

In [22]:
#Assign labels to the blockedpairs df
ban_half_pairs_a_q3_jac_03['true_label'] = assignTrueLabels(ban_half_pairs_a_q3_jac_03,'l_ban_isbn','r_half_isbn')
ban_half_pairs_fa_q3_jac_03['true_label'] = assignTrueLabels(ban_half_pairs_fa_q3_jac_03,'l_ban_isbn','r_half_isbn')
ban_half_pairs_tit_q3_jac_03['true_label'] = assignTrueLabels(ban_half_pairs_tit_q3_jac_03,'l_ban_isbn','r_half_isbn')
ban_half_pairs_titnp_q3_jac_03['true_label'] = assignTrueLabels(ban_half_pairs_titnp_q3_jac_03,'l_ban_isbn','r_half_isbn')
ban_half_pairs_titnp_nosw_q3_jac_03['true_label'] = assignTrueLabels(ban_half_pairs_titnp_nosw_q3_jac_03,'l_ban_isbn','r_half_isbn')

In [23]:
print('Blocking based on author, Jaccaed, 3-gram, Threshold 0.3')
print('Evaluation of ban_half_pairs_a_q3_jac_03')
print('Number of rows: {:,}'.format(ban_half_pairs_a_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,half,ban_half_pairs_a_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_half,ban_half_pairs_a_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of ban_half_pairs_fa_q3_jac_03')
print('Number of rows: {:,}'.format(ban_half_pairs_fa_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,half,ban_half_pairs_fa_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_half,ban_half_pairs_fa_q3_jac_03,'true_label')))
print('\n')
print('Blocking based on title')

print('Evaluation of ban_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_half_pairs_tit_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,half,ban_half_pairs_tit_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_half,ban_half_pairs_tit_q3_jac_03,'true_label')))
print('\n')
print('Evaluation of ban_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_half_pairs_titnp_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,half,ban_half_pairs_titnp_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_half,ban_half_pairs_titnp_q3_jac_03,'true_label')))

print('\n')
print('Evaluation of ban_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)')
print('Number of rows: {:,}'.format(ban_half_pairs_titnp_nosw_q3_jac_03.shape[0]))
print('Reduction Ratio: {}%'.format(calcRedRatio(ban,half,ban_half_pairs_titnp_nosw_q3_jac_03)))
print('Pair Completeness: {}%'.format(calcPairCompISBN(matches_ban_half,ban_half_pairs_titnp_nosw_q3_jac_03,'true_label')))

Blocking based on author, Jaccaed, 3-gram, Threshold 0.3
Evaluation of ban_half_pairs_a_q3_jac_03
Number of rows: 49,764
Reduction Ratio: 99.82%
Pair Completeness: 99.21%


Evaluation of ban_half_pairs_fa_q3_jac_03
Number of rows: 51,047
Reduction Ratio: 99.81%
Pair Completeness: 99.21%


Blocking based on title
Evaluation of ban_half_pairs_tit_q3_jac_03 (Title, Jaccaed, 3-gram, Threshold 0.3)
Number of rows: 3,493
Reduction Ratio: 99.99%
Pair Completeness: 87.47%


Evaluation of ban_half_pairs_titnp_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 1,094
Reduction Ratio: 100.0%
Pair Completeness: 52.71%


Evaluation of ban_half_pairs_titnp_nosw_q3_jac_03 (Title excluded text in parenthesis, Jaccard, 3-gram, Threshold 0.3)
Number of rows: 1,094
Reduction Ratio: 100.0%
Pair Completeness: 52.71%


In [24]:
ban_half_pairs_a_q3_jac_03.to_csv('filtered_correspondences/ban_half_pairs_a_q3_jac_03.csv',index=False)

In [89]:
matches_ban_half.to_csv('matching_pairs/matches_ban_half.csv',index=False)

-- Blocking based on Jaccard distance with 3-gram tokens on author (ban_author, half_author) performs the best. Better than taking ban_firstauthors and half_author.